# LAB 03 - Data preprocessing

## 06 시계열 데이터 처리

### #01.준비작업

#### 1-1 라이브러리 불러오기

In [1]:
from hossam import load_data
from pandas import to_datetime, DateOffset
import datetime as dt
from datetime import datetime as dt
from datetime import timedelta as td

#### 1-2 데이터 불러오기

In [2]:
origin = load_data ('shopping_orders')
origin.head()

[data] https://data.hossam.kr/data/lab03/shopping_orders.xlsx
[desc] 어느 쇼핑몰의 고객별 주문 내역 데이터
[!] Cannot read metadata


,고객명,주문일,배송완료일시,상품명,상품분류,주문금액,결제방법,배송지역
고객ID,,,,,,,,
C001,김민수,2024년 1월 15일,2024-01-17 14:30:00,무선 이어폰,전자기기,89000,카드결제,서울
C002,이영희,2024년 2월 3일,2024-02-05 16:20:00,겨울 코트,의류,156000,계좌이체,부산
C003,박철수,2024년 2월 14일,2024-02-16 11:45:00,발렌타인 초콜릿,식품,25000,카드결제,대구
C004,최지영,2024년 3월 8일,2024-03-10 13:15:00,여성 향수,화장품,78000,카드결제,인천
C005,정한솔,2024년 3월 22일,2024-03-24 09:30:00,게이밍 마우스,전자기기,65000,카드결제,서울


#### 1-3 데이터 타입 확인
- 주문일 컬럼은 문자열, 배송 완료 일시 컬럼은 날짜 타입

In [3]:
origin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, C001 to C018
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   고객명     30 non-null     object        
 1   주문일     30 non-null     object        
 2   배송완료일시  30 non-null     datetime64[ns]
 3   상품명     30 non-null     object        
 4   상품분류    30 non-null     object        
 5   주문금액    30 non-null     int64         
 6   결제방법    30 non-null     object        
 7   배송지역    30 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.1+ KB


### #02.문자열을 날짜로 변환
- Pandas 의 to_datetime() 함수는 문자열을 datetime 객체로 변환
- 날짜 형식을 자동으로 인식하지만, format 파라미터로 형식 지정도 가능함

In [5]:
df = origin.copy()  #원본 데이터 보호를 위해 복사본 생성


# 문자열에 대한 날짜 타입 변환
# --> 년,월,일에 대한 format 지정 필요
df['주문일_타입변환'] = to_datetime(df['주문일'],format='%Y년 %m월 %d일')


#변환 전,후의 데이터 타입 확인
print(df[['주문일','주문일_타입변환']].info())

#변환/전 후의 데이터 확인
df[['주문일','주문일_타입변환']].head()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, C001 to C018
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   주문일       30 non-null     object        
 1   주문일_타입변환  30 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 720.0+ bytes
None


,주문일,주문일_타입변환
고객ID,,
C001,2024년 1월 15일,2024-01-15
C002,2024년 2월 3일,2024-02-03
C003,2024년 2월 14일,2024-02-14
C004,2024년 3월 8일,2024-03-08
C005,2024년 3월 22일,2024-03-22


### #03.날짜 타입 데이터에서 정보 추출
- 날짜 타입의 컬럼은 .dt 속성을 사용하여 다양한 정보를 추출할 수 있다
- .dt.year : 년도 추출
- .dt.month : 월 추출
- .dt.day : 일 추출

#### 3-1 년/월/일 정보 추출

In [7]:
# 주문일에서 년/월/일 정보 추출

df['주문년도'] = df['주문일_타입변환'].dt.year
df['주문월']= df['주문일_타입변환'].dt.month
df['주문일자']= df['주문일_타입변환'].dt.day



#결과 확인
df[['고객명','주문일','주문년도','주문월','주문일자']].head()

,고객명,주문일,주문년도,주문월,주문일자
고객ID,,,,,
C001,김민수,2024년 1월 15일,2024,1,15
C002,이영희,2024년 2월 3일,2024,2,3
C003,박철수,2024년 2월 14일,2024,2,14
C004,최지영,2024년 3월 8일,2024,3,8
C005,정한솔,2024년 3월 22일,2024,3,22


#### 3-2 요일 정보 추출
- dt.day_name() : 요일을 영어 이름으로 반환
- dt.dayofweek 요일을 숫자로 변환 (0=월요일 , 6 = 일요일)

In [8]:
#주문일 기준 요일 정보 추출
df['주문요일영문'] = df['주문일_타입변환'].dt.day_name()
df['주문요일번호']=df['주문일_타입변환'].dt.dayofweek #0 은 월요일, 6을 일요일




#한글 요일명 매필
요일매핑 = {0: '월요일', 1: '화요일', 2: '수요일', 3: '목요일',4: '금요일', 5: '토요일', 6: '일요일'}


df['주문요일'] = df['주문요일번호'].map(요일매핑)


#결과 확인
df[['고객명','주문일','주문요일번호','주문요일영문','주문요일']].head()

,고객명,주문일,주문요일번호,주문요일영문,주문요일
고객ID,,,,,
C001,김민수,2024년 1월 15일,0,Monday,월요일
C002,이영희,2024년 2월 3일,5,Saturday,토요일
C003,박철수,2024년 2월 14일,2,Wednesday,수요일
C004,최지영,2024년 3월 8일,4,Friday,금요일
C005,정한솔,2024년 3월 22일,4,Friday,금요일


#### 3-3 분기와 기타 정보 추출


- .dt.quarter : 분기 정보 (1, 2, 3, 4)
- .dt.dayofyear : 1년 중 몇 번째 날인지에 대한 값
- .dt.isocalendar().week : 1년 중 몇 번째 주인지에 대한 값

In [9]:
# 주문일 기준 분기 및 기타 정보 추출
df['주문분기'] = df['주문일_타입변환'].dt.quarter 
df['연중몇일째'] = df['주문일_타입변환'].dt.dayofyear  #1년 중 몇 번째 날
df['연중몇주째'] = df['주문일_타입변환'].dt.isocalendar().week #1년 중 몇 번째 주 


#결과 확인
df[['고객명','주문일','주문분기','연중몇일째','연중몇주째']].head()

,고객명,주문일,주문분기,연중몇일째,연중몇주째
고객ID,,,,,
C001,김민수,2024년 1월 15일,1,15,3
C002,이영희,2024년 2월 3일,1,34,5
C003,박철수,2024년 2월 14일,1,45,7
C004,최지영,2024년 3월 8일,1,68,10
C005,정한솔,2024년 3월 22일,1,82,12


### #04.날짜 계산하기

- 두 날짜 간 차이: (날짜2 - 날짜1).dt.days
- 현재 날짜와의 차이: pd.Timestamp.today()와 비교
- 결과는 일 단위로 출력

#### 4-1 날짜 간 차이 계산

In [12]:
# 배공 소요일 계산 (배송완료일 - 주문일)
df['배송소요일'] = (df['배송완료일시'] - df['주문일_타입변환']).dt.days


#현재 날짜 기준 주문 후 경과일 계산
오늘 = dt.now()
df['배송완료후_경과일'] = (오늘 - df['배송완료일시']).dt.days

df[['주문일','배송완료일시','배송소요일','배송완료후_경과일']].head()

,주문일,배송완료일시,배송소요일,배송완료후_경과일
고객ID,,,,
C001,2024년 1월 15일,2024-01-17 14:30:00,2,683
C002,2024년 2월 3일,2024-02-05 16:20:00,2,664
C003,2024년 2월 14일,2024-02-16 11:45:00,2,653
C004,2024년 3월 8일,2024-03-10 13:15:00,2,630
C005,2024년 3월 22일,2024-03-24 09:30:00,2,617


#### 4-2 기준일로부터 년/월/일 더하기 뺴기
- 파이썬 내장 API 사용 (datetime.Timedelta(days=n)) : n 일 더하거나 뺴기

- Pandas API 사용 
pd.DateOffset(years=y, months=m, days=d, hours=h, minutes=i, seconds=s)

In [14]:
# 재주문 예상일 (배송 완료후 30일)
df['재주문예상일'] = df['배송완료일시'] + td(days=30)

#1년 A/S 보장 종료일 (주문일 기준)
df['AS보장만료일'] = df['주문일_타입변환'] + DateOffset(years=1)

# 예쌍 매출 집계일 (주문일에서 3개월 후)
df['매출집계예정일'] = df['주문일_타입변환'] + DateOffset(months=3)

# 리뷰 요청 예정일 (배송 완료 시점에서 7일 12시간 경과 후)
df['리뷰요청예정일시'] = df['배송완료일시'] + DateOffset(days=3,hours=12)

#결과 확인

df[['고객명','주문일','배송완료일시','재주문예상일','AS보장만료일','매출집계예정일','리뷰요청예정일시']].head()

,고객명,주문일,배송완료일시,재주문예상일,AS보장만료일,매출집계예정일,리뷰요청예정일시
고객ID,,,,,,,
C001,김민수,2024년 1월 15일,2024-01-17 14:30:00,2024-02-16 14:30:00,2025-01-15,2024-04-15,2024-01-21 02:30:00
C002,이영희,2024년 2월 3일,2024-02-05 16:20:00,2024-03-06 16:20:00,2025-02-03,2024-05-03,2024-02-09 04:20:00
C003,박철수,2024년 2월 14일,2024-02-16 11:45:00,2024-03-17 11:45:00,2025-02-14,2024-05-14,2024-02-19 23:45:00
C004,최지영,2024년 3월 8일,2024-03-10 13:15:00,2024-04-09 13:15:00,2025-03-08,2024-06-08,2024-03-14 01:15:00
C005,정한솔,2024년 3월 22일,2024-03-24 09:30:00,2024-04-23 09:30:00,2025-03-22,2024-06-22,2024-03-27 21:30:00
